In [1]:
import numpy as np
#
# The function in this file follow closely after the MATLAB scripts
# written by Alessandro Foi and Markku Makitalo - Tampere University
# of Technology - 2011-2012
#
#    http://www.cs.tut.fi/~foi/invansc/
#
# that accompanies their paper:
#
#    Reference: M. Makitalo and A. Foi, "Optimal inversion of the
#    generalized Anscombe transformation for Poisson-Gaussian noise",
#    IEEE Trans. Image Process., doi:10.1109/TIP.2012.2202675


#-------- Variance stabilizing transforms -----------

def anscombe(x):
    '''
    Compute the anscombe variance stabilizing transform.
      the input   x   is noisy Poisson-distributed data
      the output  fx  has variance approximately equal to 1.
    Reference: Anscombe, F. J. (1948), "The transformation of Poisson,
    binomial and negative-binomial data", Biometrika 35 (3-4): 246-254
    '''
    return 2.0*np.sqrt(x + 3.0/8.0)

def inverse_anscombe(z):
    '''
    Compute the inverse transform using an approximation of the exact
    unbiased inverse.
    Reference: Makitalo, M., & Foi, A. (2011). A closed-form
    approximation of the exact unbiased inverse of the Anscombe
    variance-stabilizing transformation. Image Processing.
    '''
    #return (z/2.0)**2 - 3.0/8.0
    return (1.0/4.0 * np.power(z, 2) +
            1.0/4.0 * np.sqrt(3.0/2.0) * np.power(z, -1.0) -
            11.0/8.0 * np.power(z, -2.0) + 
            5.0/8.0 * np.sqrt(3.0/2.0) * np.power(z, -3.0) - 1.0 / 8.0)

def generalized_anscombe(x, mu, sigma, gain=1.0):
    '''
    Compute the generalized anscombe variance stabilizing transform,
    which assumes that the data provided to it is a mixture of poisson
    and gaussian noise.
    The input signal  z  is assumed to follow the Poisson-Gaussian noise model
        x = gain * p + n
    where gain is the camera gain and mu and sigma are the read noise
    mean and standard deviation.
    We assume that x contains only positive values.  Values that are
    less than or equal to 0 are ignored by the transform.
    Note, this transform will show some bias for counts less than
    about 20.
    '''
    y = gain*x + (gain**2)*3.0/8.0 + sigma**2 - gain*mu

    # Clamp to zero before taking the square root.
    return (2.0/gain)*np.sqrt(np.maximum(y, 0.0))

def inverse_generalized_anscombe(x, mu, sigma, gain=1.0):
    '''
    Applies the closed-form approximation of the exact unbiased
    inverse of Generalized Anscombe variance-stabilizing
    transformation.
    The input signal x is transform back into a Poisson random variable
    based on the assumption that the original signal from which it was
    derived follows the Poisson-Gaussian noise model:
        x = gain * p + n
    where gain is the camera gain and mu and sigma are the read noise
    mean and standard deviation.
    Roference: M. Makitalo and A. Foi, "Optimal inversion of the
    generalized Anscombe transformation for Poisson-Gaussian noise",
    IEEE Trans. Image Process., doi:10.1109/TIP.2012.2202675
    '''
    test = np.maximum(x, 1.0)
    exact_inverse = ( np.power(test/2.0, 2.0) +
                      1.0/4.0 * np.sqrt(3.0/2.0)*np.power(test, -1.0) -
                      11.0/8.0 * np.power(test, -2.0) +
                      5.0/8.0 * np.sqrt(3.0/2.0) * np.power(test, -3.0) -
                      1.0/8.0 - np.power(sigma, 2) )
    exact_inverse = np.maximum(0.0, exact_inverse)
    exact_inverse *= gain
    exact_inverse += mu
    exact_inverse[np.where(exact_inverse != exact_inverse)] = 0.0
    return exact_inverse

#def inverse_generalized_anscombe(y,mu,sigma,gain=1.0):
#    return (1.0/gain)*(gain*y/2.0)**2 - gain*3.0/8.0 - (sigma**2)/gain + mu

In [2]:
from bm3d import bm3d, BM3DProfile
import numpy as np
import scipy.misc
from matplotlib import pyplot as plt
from tifffile import imread, imsave

In [6]:
path='/Users/prakash/Downloads/Convallaria_diaphragm/'

# Load the test data
noisy=imread(path+"20190520_tl_25um_50msec_05pc_488_130EM_Conv.tif")[:,:512,:512].astype(np.float32)
# We are loading only a sub image to spped up computation

# We estimate the ground truth by averaging.
clean=np.mean(noisy[:,...],axis=0)[np.newaxis,...]

print(noisy.shape)
print(clean.shape)


(100, 512, 512)
(1, 512, 512)


In [7]:
def PSNR(gt, img, rangePSNR):
    '''
    Compute PSNR.
    Parameters
    ----------
    gt: array
        Ground truth image.
    img: array
        Predicted image.''
    '''
    mse = np.mean(np.square(gt - img))
    return 20 * np.log10(rangePSNR) - 10 * np.log10(mse)

In [ ]:
offset=0
from tqdm import tqdm
psnrs=[]

for img_idx in range(noisy.shape[0]):
    print("--------------------imgae:", img_idx, " ------------------------------------")
    denoisedImgs=[]
    mses=[]
    bestMSE = None
    bestImg = None
    bestSig= None
    
    for s in tqdm(range(1)):
#         sig = (s+1)*2.5
        sig = 3*2.5

        img=noisy[img_idx,...]
        cl= clean[0]

        
        ans=anscombe(img-offset)
        denoised = inverse_anscombe (bm3d (ans,sig)) + offset


        mse = np.mean((cl-denoised)**2)
        mses.append(mse)
        denoisedImgs.append(denoised.copy())

        if bestMSE is None or (mse<bestMSE):
            bestImg = denoised.copy()
            bestMSE = mse
            bestSig = sig
        
    rangePSNR = np.max(cl)-np.min(cl)
    psnr = PSNR(cl, bestImg, rangePSNR)
    psnrs.append(psnr)
    print("image:", img_idx, "\t PSNR:", psnr, "\t mean PSNR:", np.mean(psnrs))


--------------------imgae: 0  ------------------------------------


100%|██████████| 1/1 [00:07<00:00,  7.04s/it]


image: 0 	 PSNR: 35.331679559868505 	 mean PSNR: 35.331679559868505
--------------------imgae: 1  ------------------------------------


100%|██████████| 1/1 [00:08<00:00,  8.24s/it]


image: 1 	 PSNR: 35.30759272805587 	 mean PSNR: 35.319636143962185
--------------------imgae: 2  ------------------------------------


...

100%|██████████| 1/1 [00:07<00:00,  7.53s/it]


image: 45 	 PSNR: 35.432764679895705 	 mean PSNR: 35.4243325737835
--------------------imgae: 46  ------------------------------------


100%|██████████| 1/1 [00:05<00:00,  5.66s/it]


image: 46 	 PSNR: 35.380729763835 	 mean PSNR: 35.42340485442289
--------------------imgae: 47  ------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]